In [ ]:
import numpy as np
from scipy.io.wavfile import read as audioread
from scipy.io import loadmat
from scipy.stats import zscore
from scipy.signal import hilbert, resample_poly, correlate, find_peaks, peak_prominences
from nilearn.image import load_img, coord_transform
from brainvoyagertools.voi import VOIsDefinition as voi_reader
import matplotlib.pyplot as plt
import mplcursors
%matplotlib qt

load Pie-man story

In [ ]:
Fs, y = audioread('../fMRI Data for Tutorial and HW/Story_Original_MRI.wav')
N = y.shape[0]
slength = N/Fs
time = np.linspace(0, N/Fs, N)
signal = np.mean(y, 1)
signal_nor = zscore(signal)

Envelope  
  
there is no direct equvilent to MATLAB's envelope  
Doing my best to replicate envelope(signal_nor,Fs/10,'peak');

In [ ]:
window_size = Fs // 10
# Find peaks
peaks, _ = find_peaks(signal_nor, distance=window_size)
# Calculate the prominences of the peaks
prominences = peak_prominences(signal_nor, peaks)[0]
# Interpolate the upper and lower envelopes
envelope = np.interp(np.arange(len(signal_nor)), peaks, prominences)

Resample

In [ ]:
T1 = resample_poly(envelope, 1, Fs)
T = resample_poly(T1, 2, 3)
T = T[9:290]
T = np.convolve(T, np.ones(5)/5, mode='valid')  # mimicks smooth, actually better results without

fMRI data - extract average ROI

In [ ]:
roinii1 = load_img('../fMRI Data for Tutorial and HW/MNI152_T1_3mm_brain.nii')
voi = voi_reader(load='../fMRI Data for Tutorial and HW/a1_group2_new.voi')

Convert VOI to Voxels

In [ ]:
inv_affine_mat = np.linalg.inv(roinii1.affine)
voxels = []
for x, y, z in voi.vois[4].data:
    x_vox, y_vox, z_vox = coord_transform(x, y, z, inv_affine_mat)
    voxels.append(np.round((x_vox, y_vox, z_vox)).astype(int))
voxels = np.array(voxels)

Create binay mask

In [ ]:
roi = np.zeros(roinii1.shape, dtype=int)
for voxel in voxels:
    roi[voxel[0], voxel[1], voxel[2]] = 1

reshape the 3d matrix to 1D vector (roimask)

In [ ]:
roimask = roi.flatten('F')

In [ ]:
data = loadmat('../fMRI Data for Tutorial and HW/story_selected_reg_wm_csf_hsd_last_half2/avg/A0.mat')
data = data['bold_avg']

In [ ]:
ROI = data[:, roimask == 1]
ROI_nor = zscore(ROI)
mean_ROI = np.mean(ROI_nor, 1)
mean_ROI = zscore(mean_ROI)

cross correlation

In [ ]:
correlation = correlate(T, mean_ROI, mode='full')
# Normalize the cross-correlation
r = correlation / (np.linalg.norm(T) * np.linalg.norm(mean_ROI))
# Determine the lag values
lag = np.arange(-(len(T) - 1), len(mean_ROI))

In [ ]:
fig, ax = plt.subplots()
ax.plot(lag, r)
ax.set_ylabel('Magnitude')
ax.set_xlabel('Time')
ax.set_title('Cross Correlation Between BOLD fMRI & Envelope Audio')
plt.show()